# Regression logistique 

Nous souhaitons ici modéliser, classfier, "is_genuine",une variable binaire [0,1] en fonction des variables explicatives quantitatives 
(et potentiellement quqlitatives) à l'aide d'une regression logistique.
La regression logistique est une méthode de classification supervisée.

L'objectif ici est de modéliser la probabilité d'obtenir des vrai ou faux billet de banque en fonction des constantes des billets.


La variable is_genuine, qui prend comme valeurs 1 ou  0, suit une loi de Bernoulli de paramètre p(x) dépendant de x

Le Machine Learning consiste à nourrir un algorithme à l’aide de données afin qu’il apprenne par lui-même à effectuer une certaine tâche. Dans les problèmes de classification, il prédit des résultats que l’on doit comparer à la réalité pour mesurer son degré de performance. On utilise généralement la matrice de confusion, appelée aussi tableau de contingence. Elle mettra non seulement en valeur les prédictions correctes et incorrectes mais nous donnera surtout un indice sur le type d’erreurs commises. 

# Set up

In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pickle

from sklearn import metrics 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 1000)

import os
os.chdir('/Users/alexandrejacqueline/documents/scriptopc/P6')

# Load the data

In [8]:
with open('mypicklefile_P6.pkl','rb') as f1:
    my_unpickler = pickle.Unpickler(f1)
    notes = my_unpickler.load()
notes.head(1)

,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length,Clusters_KM
0,1,171.81,104.86,104.95,4.52,2.89,112.83,1


In [9]:
#notes = pd.read_csv('notes.csv')
notes['is_genuine'] = notes['is_genuine'].astype(int)

# Regression logistique avec SciKit Learn

In [10]:
# Définir notre variable dépendante y et nos varaibles indépendantes X
X = notes.drop(["is_genuine","Clusters_KM"],axis = 1)
y = notes.iloc[:, 0].values 

# Validation croisée 

In [11]:
# Diviser le dataset entre le Training set et le Test set
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

#Donnée standardisées
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, test_size = 0.25, random_state = 0) #Standard Scaler

#Données originelles
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)



In [12]:
#Reg_Log_0_Test
# Feature Scaling, mis ç l'échelle des données 
# Pour modèle type clustering ou notion de distance tjrs necessaire le centrage/reduction pour eviter ce problème d'échelle, histoire d'ordre de grandeur,peu importe l'unité ...

sc = StandardScaler()  #Permet de centrer reduire les variables        
X_train_0 = sc.fit_transform(X_train_0)
X_test_0 = sc.transform(X_test_0)

# Construction et entrainement de nos modèles

In [13]:
#Construction du modèle
classifier = LogisticRegression() #Création du modèle 

#Entrainement du modèle 0 Donnée standardisées
%time classifier.fit(X_train_0, y_train_0)

#Entrainement du modèle Données originelles
%time classifier.fit(X_train, y_train)

CPU times: user 161 ms, sys: 31.7 ms, total: 193 ms
Wall time: 338 ms
CPU times: user 10.4 ms, sys: 1.11 ms, total: 11.5 ms
Wall time: 11.6 ms


LogisticRegression()

In [14]:
# Faire de nouvelles prédictions, résultat des prédiction du x_test_0   / Coef de determination 
y_pred_0 = classifier.predict(X_test_0)
score=classifier.score(X_test_0,y_test_0)
print("Le score de prédiction est de {}% de taux de fiabilité".format(round(score*100)))

Le score de prédiction est de 98% de taux de fiabilité


In [15]:
# Faire de nouvelles prédictions, résultat des prédiction du x_test
y_pred = classifier.predict(X_test)
score=classifier.score(X_test,y_test)
print("Le score de prédiction est de {}% de taux de fiabilité".format(round(score*100)))

Le score de prédiction est de 100% de taux de fiabilité


La standartisation ici crée une perte de performance de notre modèle.


# Matrice de confusion 

La performance d’un algorithme de Machine Learning est directement liée à sa capacité à prédire un résultat . Lorsque l’on cherche à comparer les résultats d’un algorithme à la réalité , on utilise une matrice de confusion .

In [16]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[18  0]
 [ 0 25]]


Notre régle de décision est parfaite car les nombres de faux positifs et de faux négatifs sont nuls.(En pratique ce n'est jamais le cas)

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        25

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43



En comparaison avec notre modèle 0 

In [19]:
print(classification_report(y_test_0, y_pred_0))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97        18
           1       1.00      0.96      0.98        25

    accuracy                           0.98        43
   macro avg       0.97      0.98      0.98        43
weighted avg       0.98      0.98      0.98        43



In [25]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
print('We found that accuracy of the model is',round(accuracy_percentage),'%. By accuracy, we mean the number of correct predictions divided by the total number of predictions.')

We found that accuracy of the model is 100 %. By accuracy, we mean the number of correct predictions divided by the total number of predictions.


http://eric.univ-lyon2.fr/~ricco/cours/cours_econometrie.html

# Sauvegarde du modèle

In [14]:
with open('classifier_pred.pkl', 'wb') as f1:
    my_pickler = pickle.Pickler(f1)
    my_pickler.dump(classifier)

# Sauvegarde du dataset

In [15]:
with open('mypicklefile_P6_P2.pkl', 'wb') as f1:
    my_pickler = pickle.Pickler(f1)
    my_pickler.dump(notes)